In [144]:
#!/usr/bin/env python
import json, openai,requests
import warnings
import os
warnings.filterwarnings('ignore')
import configparser
import datetime
from time import sleep

In [2]:
readKey = configparser.ConfigParser()
readKey.read_file(open('apidata.config'))

openai_org = readKey['OPENAI']["ORG"]
openai_key = readKey['OPENAI']['KEY'] 

In [11]:
def extract_response(org,key,role_player,request_content):
    openai.organization = org
    openai.api_key= key
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", 
             "content": f"You are a {role_player}"},
            {"role": "assistant", 
             "content": f"I have assumed the role of {role_player}"},
            {"role": "user", 
             "content": f"""If you can provide the Knowledge Graph in JSONLD for
                         the dataset I provide, reply 'Yes I can.' """},
            {"role": "assistant", 
             "content": "Yes I can"},
            {"role": "user", 
             "content": f"Here is the dataset {request_content}. Provide Knowledge Graph"}
            ]
        )
    return response['choices'][0]['message']['content']

In [13]:
get_role = input("What role you want me to play: ")

What role you want me to play: Dataset Summary creator


In [8]:
get_role

'Dataset Summarizer'

In [3]:
import tiktoken
import pandas as pd
def num_tokens_from_string(string: str, 
                           encoding_name: str = 'text-davinci-003') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [5]:
#reading in the dataset sample_data.csv

with open('space_titanic.csv','r') as sd:
    data = sd.readlines()

In [8]:
data_chunks = []
rng1 = 1

In [24]:
##Version 1 of the prompt

In [20]:
for x in list(range(1,3)):
    rng2 = rng1 + 5
    print(f'before spliting data {rng1} and {rng2}')
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    print(data_t)
    prompt_made = f"""
    dataset_{x}={data_t}
    
    Prompt:Refer to the [dataset_{x}] and provide the Knowledge graph that contains the 
    details of all the columns in the dataset.    
    Answer:
    """
    data_chunks.append(prompt_made)
    print(f'Number of tokens for the dataset {x} is {num_tokens_from_string(prompt_made)}')
    rng1 = rng1 + 5

before spliting data 21 and 26
PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0017_02,Earth,False,F/6/P,55 Cancri e,14.0,False,412.0,0.0,1.0,0.0,679.0,Philda Brighttt,False\n', '0020_01,Earth,True,E/0/S,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Almary Brantuarez,False\n', '0020_02,Earth,True,E/0/S,55 Cancri e,49.0,False,0.0,0.0,0.0,0.0,0.0,Glendy Brantuarez,False\n', '0020_03,Earth,True,E/0/S,55 Cancri e,29.0,False,0.0,0.0,,0.0,0.0,Mollen Mcfaddennon,False\n', '0020_04,Earth,False,E/0/S,TRAPPIST-1e,10.0,False,0.0,0.0,0.0,0.0,0.0,Breney Jacostanley,True\n']
Number of tokens for the dataset 1 is 388
before spliting data 26 and 31
PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0020_05,Earth,True,E/0/S,PSO J318.5-22,1.0,False,,0.0,0.0,0.0,0.0,Mael Brantuarez,False\n', '0020_06,Earth,False,E/0/S,TRAPPIST-1e,7.0,False,0.0,0.0,0.0,0.0,0.0,

In [6]:
def chunk_to_df(chunk_num,data_chunk):
    
    start = data_chunk.split('\\n')
    
    clean_step1 = start[0].strip(' \n').replace('=','').replace(f'dataset_{chunk_num}','')

    clean_step2 = clean_step1.split('\n+[')
    
    header = clean_step2[0]
    row1 = clean_step2[1]
    
    clean_row = []
    for dat in start[1:]:
        clean_row.append(dat.strip(" ',"))
        
    clean_row.insert(0,row1.strip("'"))
    clean_row.insert(0,header)
    
    clean_data_list = [data.split(',') for data in clean_row]
    
    chunk_df = pd.DataFrame(clean_data_list[1:-1],columns=clean_data_list[0])
    
    return chunk_df

In [18]:
chunk_to_df(1,data_chunks[0])

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [22]:
content_output = extract_response(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=data_chunks[0])

In [23]:
content_output

'Here is the JSON-LD Knowledge Graph for the dataset you provided:\n\n```\n{\n  "@context": {\n    "schema": "http://schema.org/",\n    "passenger": "schema:Person",\n    "identifier": "schema:identifier",\n    "homePlanet": "schema:homeLocation",\n    "cryoSleep": "schema:isRelatedTo",\n    "cabin": "schema:occupationalCategory",\n    "destination": "schema:destination",\n    "age": "schema:age",\n    "vip": "schema:isVIP",\n    "roomService": "schema:hasRoomService",\n    "foodCourt": "schema:hasFoodCourt",\n    "shoppingMall": "schema:hasShoppingMall",\n    "spa": "schema:hasSpa",\n    "vrDeck": "schema:hasVRDeck",\n    "name": "schema:name",\n    "transported": "schema:isTransported"\n  },\n  "@type": "passenger",\n  "identifier": "PassengerId",\n  "homePlanet": "HomePlanet",\n  "cryoSleep": "CryoSleep",\n  "cabin": "Cabin",\n  "destination": "Destination",\n  "age": "Age",\n  "vip": "VIP",\n  "roomService": "RoomService",\n  "foodCourt": "FoodCourt",\n  "shoppingMall": "ShoppingMa

In [ ]:
## Version 2 of the prompt

In [28]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset_{x}={data_t}

Prompt:Refer to the [dataset] and summarize it as a Knowledge graph that contains the 
details of all the columns in the dataset, and your inference of the dataset    
Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 31 and 36
Number of tokens for the dataset is 397

dataset_2=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0026_01,Europa,False,C/0/P,55 Cancri e,34.0,False,22.0,6073.0,0.0,1438.0,328.0,Anyoni Unconary,False\n', '0028_01,Mars,False,F/8/P,TRAPPIST-1e,43.0,False,1125.0,0.0,136.0,48.0,0.0,Ceros Mare,False\n', '0030_01,Earth,False,G/4/S,TRAPPIST-1e,32.0,False,0.0,850.0,81.0,437.0,453.0,Ginia Morsentley,False\n', '0031_01,Mars,False,F/9/P,TRAPPIST-1e,47.0,False,214.0,0.0,1411.0,0.0,1229.0,Coobix Datie,True\n', '0031_02,Mars,False,F/9/P,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,0.0,0.0,Cinets Datie,True\n']

Prompt:Refer to the [dataset] and summarize it as a Knowledge graph that contains the 
details of all the columns in the dataset, and your inference of the dataset    
Answer:



In [30]:
content_output_2 = extract_response(org=openai_org,key=openai_key,
                                role_player=get_role, 
                                  request_content=prompt_made)

In [31]:
content_output_2

'Here\'s the Knowledge Graph in JSON-LD format for the given dataset:\n\n```\n{\n  "@context": {\n    "schema": "http://schema.org/"\n  },\n  "@type": "schema:Dataset",\n  "schema:name": "Passenger Information",\n  "schema:description": "A dataset containing information about passengers travelling to TRAPPIST-1e from various planets.",\n  "schema:creator": "Your Name",\n  "schema:variableMeasured": {\n    "@type": "schema:Property",\n    "schema:name": "Passenger Attributes",\n    "schema:description": "The attributes of each passenger in the dataset.",\n    "schema:variableMeasured": ["HomePlanet", "CryoSleep", "Cabin", "Destination", "Age", "VIP", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Name", "Transported"]\n  },\n  "schema:includedInDataCatalog": {\n    "@type": "schema:DataCatalog",\n    "schema:name": "Some Dataset Repository",\n    "schema:url": "http://example.com"\n  }\n}\n```\n\nFrom the given dataset, we can infer that the dataset contains information a

In [32]:
#version 3 of the prompt

In [17]:
def extract_summary(org,key,role_player,request_content):
    openai.organization = org
    openai.api_key= key
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", 
             "content": f"You are a {role_player}"},
            {"role": "assistant", 
             "content": f"I have assumed the role of {role_player}"},
            {"role": "user", 
             "content": f"Here is the dataset {request_content}."}
            ]
        )
    return response['choices'][0]['message']['content']

In [9]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and list 50 statistical, factual and inferential questions    
Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 1 and 6
Number of tokens for the dataset is 377

dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False\n', '0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True\n', '0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False\n', '0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False\n', '0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True\n']

Prompt:Refer to the [dataset] and list 50 statistical, factual and inferential questions    
Answer:



In [18]:
inference_3 = extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [20]:
inference_3

"1. How many entries are in the dataset?\n2. What is the minimum age of the passengers?\n3. What is the maximum age of the passengers?\n4. What is the average age of the passengers?\n5. What is the median age of the passengers?\n6. What is the mode of the age distribution of the passengers?\n7. How many different home planets are represented in the dataset?\n8. What is the most common home planet?\n9. How many passengers are in cryosleep?\n10. What percentage of the passengers are in cryosleep?\n11. What is the distribution of cabins among the passengers?\n12. What is the most common cabin type?\n13. What is the most common destination?\n14. How many passengers are VIPs?\n15. What percentage of the passengers are VIPs?\n16. What is the average room service rating of the passengers?\n17. What is the median room service rating of the passengers?\n18. What is the mode of the room service ratings of the passengers?\n19. What is the average food court rating of the passengers?\n20. What is 

In [21]:
questions_list = inference_3.split('\n')

In [26]:
questions_list[6:25]

['7. How many different home planets are represented in the dataset?',
 '8. What is the most common home planet?',
 '9. How many passengers are in cryosleep?',
 '10. What percentage of the passengers are in cryosleep?',
 '11. What is the distribution of cabins among the passengers?',
 '12. What is the most common cabin type?',
 '13. What is the most common destination?',
 '14. How many passengers are VIPs?',
 '15. What percentage of the passengers are VIPs?',
 '16. What is the average room service rating of the passengers?',
 '17. What is the median room service rating of the passengers?',
 '18. What is the mode of the room service ratings of the passengers?',
 '19. What is the average food court rating of the passengers?',
 '20. What is the median food court rating of the passengers?',
 '21. What is the mode of the food court ratings of the passengers?',
 '22. What is the average shopping mall rating of the passengers?',
 '23. What is the median shopping mall rating of the passengers?

In [72]:
quest_string = ''
for string in questions_list[6:15]:
    quest_string = quest_string + string.split('.')[1]

In [39]:
###Follow up prompt

In [73]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and answer succinctly to the following questions. 
{quest_string}
At the end print COMPLETED INFERENCE

Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 41 and 46
Number of tokens for the dataset is 468

dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0039_01,Earth,True,G/1/P,55 Cancri e,30.0,False,0.0,0.0,,0.0,0.0,Jorgie Batthewitt,False\n', '0041_01,Earth,True,G/2/P,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,0.0,0.0,Margia Moodsey,True\n', '0043_01,Europa,False,B/3/P,TRAPPIST-1e,45.0,False,0.0,164.0,45.0,2511.0,855.0,Ankalik Cylistrand,False\n', '0044_01,Earth,True,G/3/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Jodye Coopelandez,False\n', '0044_02,Earth,True,G/3/P,55 Cancri e,4.0,False,0.0,0.0,0.0,0.0,0.0,Kayne Coopelandez,True\n']

Prompt:Refer to the [dataset] and answer succinctly to the following questions. 
 How many different home planets are represented in the dataset? What is the most common home planet? How many passengers are in cryosleep? What percentage of the passengers are in cryosleep? What is the distribution of cabins 

In [74]:
designed_summary =  extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [75]:
designed_summary

'- There are at least 3 different home planets represented in the dataset.\n- The most common home planet is Earth, as it appears at least twice in the dataset.\n- There are at least 2 passengers in cryosleep.\n- It is not possible to calculate the percentage of passengers that are in cryosleep as the total number of passengers is not provided in the dataset.\n- The cabins distribution is partially represented in the dataset, with at least 4 different cabin types: G/1/P, G/2/P, B/3/P and G/3/P.\n- The most common cabin type is G/3/P, as it appears at least twice in the dataset.\n- TRAPPIST-1e is the most common destination, as it appears at least 3 times in the dataset.\n- There are no VIP passengers in the dataset.\n- It is not possible to calculate the percentage of VIP passengers as the total number of passengers is not provided in the dataset.\n\nCOMPLETED INFERENCE'

In [76]:
num_tokens_from_string(designed_summary)

216

In [ ]:
#check if gpt can make such inferences directly

In [77]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and succinctly list 10 statistical, factual and inferences    
Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 41 and 46
Number of tokens for the dataset is 377

dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0039_01,Earth,True,G/1/P,55 Cancri e,30.0,False,0.0,0.0,,0.0,0.0,Jorgie Batthewitt,False\n', '0041_01,Earth,True,G/2/P,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,0.0,0.0,Margia Moodsey,True\n', '0043_01,Europa,False,B/3/P,TRAPPIST-1e,45.0,False,0.0,164.0,45.0,2511.0,855.0,Ankalik Cylistrand,False\n', '0044_01,Earth,True,G/3/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Jodye Coopelandez,False\n', '0044_02,Earth,True,G/3/P,55 Cancri e,4.0,False,0.0,0.0,0.0,0.0,0.0,Kayne Coopelandez,True\n']

Prompt:Refer to the [dataset] and succinctly list 10 statistical, factual and inferences    
Answer:



In [78]:
inference_possiblity =  extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [79]:
inference_possiblity

'1. The dataset contains information on individuals who have undergone cryosleep.\n2. Home planet, cabin, destination, and age are recorded for each individual.\n3. There are VIP individuals who have access to room service, food court, shopping mall, spa, and VR deck.\n4. Some individuals have been transported to their destination while some have not.\n5. The dataset contains information on individuals from multiple planets, including Earth and Europa.\n6. The age of individuals ranges from 4 to 55 years old.\n7. There are individuals who were transported to both TRAPPIST-1e and 55 Cancri e.\n8. The dataset contains at least 5 entries.\n9. There are individuals with similar last names (Coopelandez).\n10. There is missing data for some amenities, as some individuals have NaN values for shopping mall or spa access.'

In [54]:
#It infers in its own method. But the earlier 50 question guided it

In [88]:
inference_requests = []
rng1 = 1
for x in list(range(1,5)):
    rng2 = rng1 + 10
    print(f'before spliting data {rng1} and {rng2}')
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    #print(data_t)
    prompt_made = f"""
    dataset={data_t}

    Prompt:Refer to the [dataset] and in 300 words answer the following questions.
    Do not print the questions. 
    {quest_string}
    At the end print COMPLETED INFERENCE

    Answer:
    """
    print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
    inference_requests.append(prompt_made)
    rng1 = rng1 + 10

before spliting data 1 and 11
Number of tokens for the dataset is 775
before spliting data 11 and 21
Number of tokens for the dataset is 758
before spliting data 21 and 31
Number of tokens for the dataset is 759
before spliting data 31 and 41
Number of tokens for the dataset is 769


In [89]:
prompt_questions = []
rng1 = 1
for x in list(range(1,5)):
    rng2 = rng1 + 10
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    prompt_made = f"""
    dataset={data_t}

    Prompt:Refer to the [dataset] and make the phy-beta-gamma version of inferences.

    Answer:
    """
    print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
    prompt_questions.append(prompt_made)
    rng1 = rng1 + 10

Number of tokens for the dataset is 678
Number of tokens for the dataset is 661
Number of tokens for the dataset is 662
Number of tokens for the dataset is 672


In [108]:
prompt_to_pbg = []
rng1 = 41
for x in list(range(1,5)):
    rng2 = rng1 + 10
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    prompt_made = f"""
    dataset={data_t}

    Prompt:Refer to the [dataset] and make the phy-beta-gamma version of inferences.

    Answer:
    """
    print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
    prompt_to_pbg.append(prompt_made)
    rng1 = rng1 + 10

Number of tokens for the dataset is 663
Number of tokens for the dataset is 663
Number of tokens for the dataset is 666
Number of tokens for the dataset is 679


In [110]:
print(prompt_to_pbg[0])


    dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0039_01,Earth,True,G/1/P,55 Cancri e,30.0,False,0.0,0.0,,0.0,0.0,Jorgie Batthewitt,False\n', '0041_01,Earth,True,G/2/P,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,0.0,0.0,Margia Moodsey,True\n', '0043_01,Europa,False,B/3/P,TRAPPIST-1e,45.0,False,0.0,164.0,45.0,2511.0,855.0,Ankalik Cylistrand,False\n', '0044_01,Earth,True,G/3/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Jodye Coopelandez,False\n', '0044_02,Earth,True,G/3/P,55 Cancri e,4.0,False,0.0,0.0,0.0,0.0,0.0,Kayne Coopelandez,True\n', '0044_03,Earth,True,G/3/P,PSO J318.5-22,21.0,False,0.0,0.0,0.0,0.0,0.0,Cassa Coopelandez,True\n', '0045_01,Mars,False,F/10/P,TRAPPIST-1e,21.0,False,970.0,0.0,180.0,0.0,64.0,Zelowl Chmad,False\n', '0045_02,Mars,True,F/10/P,,19.0,False,0.0,0.0,0.0,0.0,0.0,Mass Chmad,True\n', '0050_01,Earth,False,E/1/S,55 Cancri e,35.0,False,790.0,0.0,0.0,,0.0,Sony Lancis,False\n', '0051_01,Ea

In [85]:
len(inference_requests)

4

In [90]:
for ind,inf in enumerate(inference_requests):
    comp = extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=inf)
    prompt_dict = {'prompt':prompt_questions[ind],
                  'completion':comp}
    with open('inferences_prompt_04.jsonl','a+') as inje:
        json.dump(prompt_dict,fp=inje)
        inje.write('\n')

In [91]:
num_tokens_from_string(prompt_dict['completion'])

346

In [158]:
def file_upload(filename, purpose='fine-tune'):
    resp = openai.File.create(purpose=purpose,file=open(file=filename,
                                                        mode='rb'))
    return resp["id"]

def finetune_model(fileId, suffix,model='davinci',api_key=openai_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    payload = {'training_file':fileId,'model':model,'suffix':suffix}
    resp = requests.request(method='POST',
                            url='https://api.openai.com/v1/fine-tunes',
                           headers=header,
                           json=payload,timeout=40)
    model_id = resps.json()['id']
    return(model_id)
    
def finetune_get(ftId,api_key=openai_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}',
                           headers=header,timeout=40)
    reply_matter = resp.json()['events']
    return(reply_matter)
    
def finetune_event(ftId,api_key=openai_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}/events',
                           headers=header,timeout=40)
    print(resp.json())

In [154]:
def gpt3_completion(org, key,prompt,prompt_part,
                    engine='text-davinci-003',temp=0.1,
                   top_p=1.0,tokens=200,freq_pen=0.0,
                   pres_pen=0.0,stop=['asdfasdf','asdfasdf'],
                   filename='inferences_prompt_06.jsonl'):
    """The function will take the prompt and the prompt parts to 
    begin the data collection. Once the data collection is completed, 
    the prompt part will be used for fine tuning. The prompt used will 
    be discarded"""
    openai.api_key = key
    openai.organization = org
    max_retry=5
    retry = 0
    prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()
    while True:
        try:
            response = openai.Completion.create(
            model=engine,
            prompt=prompt,
            temperature=temp,
            max_tokens=tokens,
            top_p=top_p,
            frequency_penalty=freq_pen,
            presence_penalty=pres_pen,
            stop=stop)
            
            text = response['choices'][0]['text'].strip()
            dict_reply = {'prompt':prompt_part,'completion':text}
            with open(filename,'a+') as taker:
                json.dump(dict_reply,fp=taker)
                taker.write('\n')
            
            return dict_reply
        except Exception as e:
            retry += 1
            if retry > max_retry:
                return f'GPT errored out {e}'
            sleep(1)

In [92]:
file_upload(filename='inferences_prompt_04.jsonl',)

file-eJPMsRVDjOFoLMOEditavKoj


'file-eJPMsRVDjOFoLMOEditavKoj'

In [93]:
finetune_model(fileId='file-eJPMsRVDjOFoLMOEditavKoj',
               suffix='phy-beta-gamma')

{'object': 'fine-tune', 'id': 'ft-k5AtqXOKUOouYeUsEyUsfyMO', 'hyperparams': {'n_epochs': 4, 'batch_size': None, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': None}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-eJPMsRVDjOFoLMOEditavKoj', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 9690, 'created_at': 1678494835, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [], 'created_at': 1678494855, 'updated_at': 1678494855, 'status': 'pending', 'fine_tuned_model': None, 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-k5AtqXOKUOouYeUsEyUsfyMO', 'created_at': 1678494855}]}


In [105]:
finetune_get(ftId='ft-k5AtqXOKUOouYeUsEyUsfyMO')

[{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-k5AtqXOKUOouYeUsEyUsfyMO', 'created_at': 1678494855}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Fine-tune costs $0.47', 'created_at': 1678495110}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Fine-tune enqueued. Queue number: 0', 'created_at': 1678495111}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Fine-tune started', 'created_at': 1678495113}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Completed epoch 1/4', 'created_at': 1678495229}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Completed epoch 2/4', 'created_at': 1678495233}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Completed epoch 3/4', 'created_at': 1678495237}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Completed epoch 4/4', 'created_at': 1678495241}, {'object': 'fine-tune-event', 'level': 'info', 'message': 'Uploaded model: davinci:ft-kamal:

In [137]:
inference_requests = []
rng1 = 1
for x in list(range(1,50)):
    rng2 = rng1 + 10
    print(f'before spliting data {rng1} and {rng2}')
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    #print(data_t)
    prompt_made = f"""
    dataset={data_t}

    Prompt:Refer to the [dataset] and in 300 words answer the following questions. Remember that these queries are
    insight_builder queries.Do not print the queries. 
    {quest_string}
    At the end print COMPLETED INFERENCE

    Answer:
    """
    print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
    inference_requests.append(prompt_made)
    rng1 = rng1 + 10

before spliting data 1 and 11
Number of tokens for the dataset is 785
before spliting data 11 and 21
Number of tokens for the dataset is 768
before spliting data 21 and 31
Number of tokens for the dataset is 769
before spliting data 31 and 41
Number of tokens for the dataset is 779
before spliting data 41 and 51
Number of tokens for the dataset is 770
before spliting data 51 and 61
Number of tokens for the dataset is 770
before spliting data 61 and 71
Number of tokens for the dataset is 773
before spliting data 71 and 81
Number of tokens for the dataset is 786
before spliting data 81 and 91
Number of tokens for the dataset is 775
before spliting data 91 and 101
Number of tokens for the dataset is 771
before spliting data 101 and 111
Number of tokens for the dataset is 778
before spliting data 111 and 121
Number of tokens for the dataset is 765
before spliting data 121 and 131
Number of tokens for the dataset is 767
before spliting data 131 and 141
Number of tokens for the dataset is 76

In [141]:
prompt_questions = []
rng1 = 1
for x in list(range(1,50)):
    rng2 = rng1 + 10
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    prompt_made = f"""
    dataset={data_t}

    Prompt:Refer to the [dataset] and make inferences based on insight_builder queries.

    Answer:
    """
    print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made,encoding_name="davinci")}')
    prompt_questions.append(prompt_made)
    rng1 = rng1 + 10

Number of tokens for the dataset is 683
Number of tokens for the dataset is 666
Number of tokens for the dataset is 667
Number of tokens for the dataset is 677
Number of tokens for the dataset is 668
Number of tokens for the dataset is 668
Number of tokens for the dataset is 671
Number of tokens for the dataset is 684
Number of tokens for the dataset is 673
Number of tokens for the dataset is 669
Number of tokens for the dataset is 676
Number of tokens for the dataset is 663
Number of tokens for the dataset is 665
Number of tokens for the dataset is 662
Number of tokens for the dataset is 682
Number of tokens for the dataset is 686
Number of tokens for the dataset is 671
Number of tokens for the dataset is 679
Number of tokens for the dataset is 670
Number of tokens for the dataset is 670
Number of tokens for the dataset is 680
Number of tokens for the dataset is 671
Number of tokens for the dataset is 658
Number of tokens for the dataset is 675
Number of tokens for the dataset is 676


In [152]:
print(inference_requests[48])


    dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0517_01,Earth,False,F/109/P,TRAPPIST-1e,,False,1445.0,0.0,1.0,2.0,73.0,Jandy Kimons,False\n', '0518_01,,True,F/92/S,TRAPPIST-1e,35.0,False,0.0,0.0,0.0,0.0,0.0,Soyos Parta,True\n', '0519_01,Earth,True,G/77/S,TRAPPIST-1e,,False,0.0,0.0,0.0,0.0,0.0,Ryany Farleyatton,True\n', '0520_01,Earth,False,G/78/S,TRAPPIST-1e,55.0,False,62.0,,0.0,0.0,,Bel Mcguirez,False\n', '0521_01,Earth,False,E/32/S,TRAPPIST-1e,23.0,False,39.0,263.0,0.0,0.0,,Hery Hornettoney,True\n', '0522_01,Earth,False,E/33/P,TRAPPIST-1e,42.0,False,0.0,0.0,750.0,0.0,156.0,Glenry Parklaney,False\n', '0523_01,Earth,False,E/33/S,TRAPPIST-1e,35.0,False,553.0,213.0,1733.0,0.0,6.0,Helley Joyneidez,True\n', '0525_01,Earth,True,,TRAPPIST-1e,47.0,False,0.0,0.0,0.0,0.0,0.0,Wenise Mirezavis,True\n', '0527_01,Mars,False,F/93/S,55 Cancri e,,False,1172.0,11.0,3.0,0.0,4.0,Supce Stité,False\n', '0527_02,Mars,T

In [155]:
gpt3_completion(org=openai_org,key=openai_key,
                           filename='inferences_prompt_temp.jsonl',prompt=inference_requests[2],
                          prompt_part=prompt_questions[2])

{'prompt': "\n    dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported\n+['0017_02,Earth,False,F/6/P,55 Cancri e,14.0,False,412.0,0.0,1.0,0.0,679.0,Philda Brighttt,False\\n', '0020_01,Earth,True,E/0/S,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Almary Brantuarez,False\\n', '0020_02,Earth,True,E/0/S,55 Cancri e,49.0,False,0.0,0.0,0.0,0.0,0.0,Glendy Brantuarez,False\\n', '0020_03,Earth,True,E/0/S,55 Cancri e,29.0,False,0.0,0.0,,0.0,0.0,Mollen Mcfaddennon,False\\n', '0020_04,Earth,False,E/0/S,TRAPPIST-1e,10.0,False,0.0,0.0,0.0,0.0,0.0,Breney Jacostanley,True\\n', '0020_05,Earth,True,E/0/S,PSO J318.5-22,1.0,False,,0.0,0.0,0.0,0.0,Mael Brantuarez,False\\n', '0020_06,Earth,False,E/0/S,TRAPPIST-1e,7.0,False,0.0,0.0,0.0,0.0,0.0,Terta Mcfaddennon,False\\n', '0022_01,Mars,False,D/0/P,TRAPPIST-1e,21.0,False,980.0,2.0,69.0,0.0,0.0,,False\\n', '0024_01,Europa,True,C/2/S,TRAPPIST-1e,62.0,False,0.0,0.0,,0.0,0.0,Penton Fullided

In [156]:
for ind,inf in enumerate(inference_requests):
    comp = gpt3_completion(org=openai_org,key=openai_key,
                           filename='inferences_prompt_08.jsonl',prompt=inf,
                          prompt_part=prompt_questions[ind])
    sleep(3)

In [159]:
file_upload(filename='inferences_prompt_08.jsonl',)

'file-ElAaw9TYVGi9Rer1oyvEHsI9'

In [160]:
finetune_model(fileId='file-ElAaw9TYVGi9Rer1oyvEHsI9',
               suffix='davinci_inferences')

NameError: name 'resps' is not defined